In [ ]:
import torch
import json
import os

# NOTE: YOU WILL HAVE TO MANUALLY SET LAYER VALUE FOR NOW
layer = 5

for folder_name in os.listdir():

    if not os.path.isdir(folder_name):
        continue

    model_name = f"{folder_name}/ae.pt"

    model = torch.load(model_name)

    hidden_dim = model['encoder.weight'].shape[0]
    input_dim = model['encoder.weight'].shape[1]

    assert hidden_dim % input_dim == 0

    expansion_factor = int(hidden_dim / input_dim)

    with open(f"{folder_name}/config.json") as f:
        config = json.load(f)


    lr = format(config['trainer']['lr'], '.0e')
    l1 = format(config['trainer']['l1_penalty'], '.0e')

    if 'activation_dim' not in config['trainer']:
        config['trainer']['activation_dim'] = input_dim
    
    if 'dictionary_size' not in config['trainer']:
        config['trainer']['dictionary_size'] = hidden_dim

    config['trainer']['layer'] = layer

    # replace the config
    with open(f"{folder_name}/config.json", 'w') as f:
        json.dump(config, f, indent=4)

    print(config)
    print(config['trainer']['lr'])
    new_folder_name = f"ef={expansion_factor}_lr={lr}_l1={l1}_layer={layer}"

    # rename the folder
    os.rename(folder_name, new_folder_name)